# 🦾 LeanAI – Preparación de Dataset y Embeddings en Google Colab

Este notebook te permite:
- Limpiar y validar tu archivo de preguntas/respuestas (Excel o CSV)
- Generar la columna `pregunta_limpia`
- Corregir filas mal formateadas
- Generar embeddings (para matching semántico)
- Descargar todo listo para usar en tu chatbot LeanAI

In [4]:
# Instala las dependencias necesarias (solo la primera vez)
!pip install -q pandas openpyxl sentence-transformers

## 📤 Sube aquí tus archivos

In [8]:
from google.colab import files
import os

print('Selecciona TODOS estos archivos para subirlos desde tu PC:')
print(' - FAQs_Ingelean.xlsx (obligatorio, Excel original)')
print(' - FAQs_Ingelean_Limpio.csv (opcional, si ya lo tienes)')
print(' - chatbot_model.pkl, tfidf_vectorizer.pkl (si quieres validar)')
uploaded = files.upload()

Selecciona TODOS estos archivos para subirlos desde tu PC:
 - FAQs_Ingelean.xlsx (obligatorio, Excel original)
 - FAQs_Ingelean_Limpio.csv (opcional, si ya lo tienes)
 - chatbot_model.pkl, tfidf_vectorizer.pkl (si quieres validar)


Saving FAQs_Ingelean.xlsx to FAQs_Ingelean (12).xlsx
Saving requirements.txt to requirements (12).txt
Saving FAQs_Ingelean_Limpio.csv to FAQs_Ingelean_Limpio (4).csv
Saving faq_embeddings.npy to faq_embeddings (10).npy
Saving LeanAI.ipynb to LeanAI (12).ipynb
Saving 02_embeddings.py to 02_embeddings (12).py
Saving 01_limpia_excel.py to 01_limpia_excel (12).py
Saving app.py to app (12).py
Saving Procfile to Procfile (12)
Saving tfidf_vectorizer.pkl to tfidf_vectorizer (12).pkl
Saving chatbot_model.pkl to chatbot_model (12).pkl


## 🚦 Limpieza y validación del archivo Excel

In [7]:
import pandas as pd
import re
from unicodedata import normalize

# Leer el Excel original
excel_name = 'FAQs_Ingelean.xlsx'
csv_name = 'FAQs_Ingelean_Limpio.csv'
try:
    df = pd.read_excel(excel_name)
    print(f'Archivo Excel {excel_name} leído correctamente.')
except Exception as e:
    print(f'No se pudo leer el Excel: {e}')
    df = None

# Mostrar columnas y primeras filas
if df is not None:
    print('Columnas:', list(df.columns))
    display(df.head())

Archivo Excel FAQs_Ingelean.xlsx leído correctamente.
Columnas: ['SUBCATEGORIA', 'PREGUNTA', 'RESPUESTA']


,SUBCATEGORIA,PREGUNTA,RESPUESTA
0,Software a la Medida,¿Qué tipo de software desarrollan?,Desarrollamos software personalizado para cont...
1,Software a la Medida,¿Me puedes explicar Qué tipo de software desar...,Desarrollamos software personalizado para cont...
2,Software a la Medida,¿Podrías mostrarme Qué tipo de software desarr...,Desarrollamos software personalizado para cont...
3,Software a la Medida,¿Me puedes contar sobre Qué tipo de software d...,Desarrollamos software personalizado para cont...
4,Software a la Medida,¿Tienen información acerca de Qué tipo de soft...,Desarrollamos software personalizado para cont...


In [9]:
# Drop the index and rename 'SUBCATEGORIA' to 'CATEGORIA'
df = df.reset_index(drop=True)
df = df.rename(columns={'SUBCATEGORIA': 'CATEGORIA'})

print('DataFrame after dropping index and renaming column:')
display(df.head())

DataFrame after dropping index and renaming column:


,CATEGORIA,PREGUNTA,RESPUESTA
0,Software a la Medida,¿Qué tipo de software desarrollan?,Desarrollamos software personalizado para cont...
1,Software a la Medida,¿Me puedes explicar Qué tipo de software desar...,Desarrollamos software personalizado para cont...
2,Software a la Medida,¿Podrías mostrarme Qué tipo de software desarr...,Desarrollamos software personalizado para cont...
3,Software a la Medida,¿Me puedes contar sobre Qué tipo de software d...,Desarrollamos software personalizado para cont...
4,Software a la Medida,¿Tienen información acerca de Qué tipo de soft...,Desarrollamos software personalizado para cont...


In [10]:
# Save the processed DataFrame to an Excel file
output_excel_name = 'FAQs_Ingelean.xlsx'
df.to_excel(output_excel_name, index=False)

# Download the Excel file
from google.colab import files
files.download(output_excel_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
# Normaliza nombres de columnas
col_map = {}
for col in df.columns:
    if col.lower().startswith('cat'): col_map[col] = 'CATEGORIA'
    if col.lower().startswith('preg'): col_map[col] = 'PREGUNTA'
    if col.lower().startswith('resp'): col_map[col] = 'RESPUESTA'
df = df.rename(columns=col_map)
df = df[['CATEGORIA', 'PREGUNTA', 'RESPUESTA']]

# Elimina filas completamente vacías
df = df.dropna(how='all')
print('Filas después de eliminar vacías:', len(df))
display(df.sample(3))

Filas después de eliminar vacías: 1991


,CATEGORIA,PREGUNTA,RESPUESTA
1814,Industria 4.0,¿Me podrías enseñar más sobre Puedo aplicar IA...,"Sí, puedes utilizar para mantenimiento predict..."
259,Administrativas,¿Cómo puedo saber más sobre Ofrecen servicios ...,"Sí, atendemos clientes en Latinoamérica y el e..."
345,Software a la Medida,¿Tienen información acerca de Pueden desarroll...,"Sí, desarrollamos apps móviles para Android y ..."


In [12]:
# Genera pregunta_limpia
def clean_text(text):
    text = str(text).lower()
    text = normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
    text = re.sub(r'[^a-z0-9\\s¿?]', '', text)
    text = re.sub(r'\\s+', ' ', text).strip()
    return text
df['pregunta_limpia'] = df['PREGUNTA'].apply(clean_text)
print('Muestra de columna pregunta_limpia:')
display(df[['PREGUNTA','pregunta_limpia']].sample(3))

Muestra de columna pregunta_limpia:


,PREGUNTA,pregunta_limpia
1077,¿Me podrías enseñar más sobre Puedo restringir...,mepodriasensenarmassobrepuedorestringirelacces...
1017,¿Dónde encuentro detalles sobre Cómo actualizo...,dondeencuentrodetallessobrecomoactualizolosdat...
49,¿Dónde encuentro detalles sobre Pueden fabrica...,dondeencuentrodetallessobrepuedenfabricardispo...


### 🚧 Verifica si hay filas rotas (mal delimitadas)

In [13]:
# Verifica filas mal formateadas al exportar a CSV (todas deben tener 4 columnas)
import csv
errores = []
with open(csv_name, 'w', encoding='utf-8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(df.columns)
    for i, row in df.iterrows():
        row_data = [row['CATEGORIA'], row['PREGUNTA'], row['RESPUESTA'], row['pregunta_limpia']]
        if any([',' in str(x) for x in row_data]):
            errores.append((i, row_data))
        writer.writerow(row_data)
if errores:
    print(f'ATENCIÓN: Hay {len(errores)} filas con comas en los textos. Revísalas si tu modelo falla.')
    display(pd.DataFrame([{'fila':i, 'data':data} for i,data in errores]).head())
else:
    print('Todas las filas están bien formateadas.')

ATENCIÓN: Hay 1606 filas con comas en los textos. Revísalas si tu modelo falla.


,fila,data
0,0,"[Software a la Medida, ¿Qué tipo de software d..."
1,1,"[Software a la Medida, ¿Me puedes explicar Qué..."
2,2,"[Software a la Medida, ¿Podrías mostrarme Qué ..."
3,3,"[Software a la Medida, ¿Me puedes contar sobre..."
4,4,"[Software a la Medida, ¿Tienen información ace..."


## 💾 Descarga el CSV limpio

In [15]:
files.download('FAQs_Ingelean_Limpio.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 🔥 Genera los embeddings para matching semántico

In [16]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
embeddings = model.encode(df['pregunta_limpia'].tolist(), convert_to_tensor=False)
np.save('faq_embeddings.npy', embeddings)
print('Embeddings generados:', embeddings.shape)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Embeddings generados: (1991, 384)


In [17]:
## 💾 Descarga los embeddings (.npy)

In [18]:
files.download('faq_embeddings.npy')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## (Opcional) Valida tus modelos `.pkl`

In [19]:
import joblib
try:
    m = joblib.load('chatbot_model.pkl')
    v = joblib.load('tfidf_vectorizer.pkl')
    print('Modelos .pkl cargados exitosamente.')
except Exception as e:
    print(f'No se pudieron cargar los modelos: {e}')

Modelos .pkl cargados exitosamente.


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.5.1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.5.1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 1.5.1 when using version 1.6.1. This might lead to breaking c